In [1]:
from keras.models import Model
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np
from time import time
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Restricting GPU memory
os.environ['CUDA_VISIBLE_DEVICES']='0'
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
import dataloader, TNN, evaluation # Loading helper functions

In [4]:
dev2train, test2train = dataloader.creating_id_mapping()

# Getting i-vectors
trn_bl_ivector, trn_bg_ivector, dev_bl_ivector, dev_bg_ivector, tst_ivector = dataloader.get_ivectors()

# Loading labels for task 2
trn_bl_id, trn_bg_id, dev_bl_id, dev_bg_id, tst_id = dataloader.get_spk_ids()
dev_trials_label = np.append(dev_bl_id,dev_bg_id)

# Making labels for task 1
trn_ivector = np.append(trn_bl_ivector, trn_bg_ivector,axis=0) # combining bg and bl speakers into a single vector
trn_trials = np.append( np.ones([len(trn_bl_ivector), 1]), np.zeros([len(trn_bg_ivector), 1]))
dev_ivector = np.append(dev_bl_ivector, dev_bg_ivector,axis=0) # combining bg and bl
dev_trials = np.append( np.ones([len(dev_bl_id), 1]), np.zeros([len(dev_bg_id), 1]))
tst_trials, tst_trials_label = dataloader.get_tst_trials()

# Models

## TNN model

In [5]:
model, anchor_input, encoded_anchor = TNN.create_TNN()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 600)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 600)          360600      anchor_input[0][0]               
                                                                 positive_input[0][0]       

## Cosine Similarity

In [6]:
from sklearn.neighbors import KNeighborsClassifier
classifier1 = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', 
                            leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=8)
classifier2 = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', 
                            leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=8)

# Prediction phase

## Getting TNN embeddings

In [7]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)
trained_model.load_weights('./weights/triplet_Dense600_87nd_err286_C1.hdf5')
transformed_trn_bl_ivector = trained_model.predict(trn_bl_ivector)
transformed_dev_ivector = trained_model.predict(dev_ivector)
transformed_trn_ivector = trained_model.predict(trn_ivector)
transformed_tst_ivector = trained_model.predict(tst_ivector)
transformed_dev_bl_ivector = trained_model.predict(dev_bl_ivector)

## Evaluating on testset

### Train on trainset only, test on testset

In [8]:
start = time()
classifier1.fit(transformed_trn_ivector, trn_trials)
task1_pred = classifier1.predict_proba(transformed_tst_ivector)
err = dataloader.calculate_EER(tst_trials,task1_pred[:,1])
print(f'time used = {time()-start:.3}')

Top S detector EER is 8.36%
time used = 1.39e+02


In [9]:
start = time()
classifier2.fit(transformed_trn_bl_ivector, trn_bl_id)
task2_pred = classifier2.predict(transformed_tst_ivector)
tst_trials_confusion = dataloader.get_trials_label_with_confusion(task2_pred, tst_trials_label, test2train, tst_trials )
dev_EER_confusion = dataloader.calculate_EER_with_confusion(task1_pred[:,1],tst_trials_confusion)
print(f'time used = {time()-start:.3}')

Top 1 detector EER is 14.08% (Total confusion error is 550)
time used = 29.4


### Train on trainset and devset, test on testset

In [10]:
# getting dev ids to train id conversion
dev_bl_id_along_trnset = []
for iter in range(len(dev_bl_id)):
    dev_bl_id_along_trnset.extend([dev2train[dev_bl_id[iter]]])

combined_embedding = np.append(transformed_trn_ivector,transformed_dev_ivector,0)
combined_trials = np.append(trn_trials,dev_trials)    
    
combined_bl_embedding = np.append(transformed_trn_bl_ivector,transformed_dev_bl_ivector,0)
combined_ids = np.append(trn_bl_id,dev_bl_id_along_trnset)

# predictions = cosine_similarity(transformed_tst_ivector, combined_embedding, combined_labels)
# err = dataloader.calculate_EER(tst_trials,predictions)

start = time()
classifier1.fit(combined_embedding, combined_trials)
task1_pred = classifier1.predict_proba(transformed_tst_ivector)
err = dataloader.calculate_EER(tst_trials,task1_pred[:,1])
print(f'time used = {time()-start:.3}')

Top S detector EER is 8.26%
time used = 1.56e+02


In [11]:
start = time()
classifier2.fit(combined_bl_embedding, combined_ids)
task2_pred = classifier2.predict(transformed_tst_ivector)
tst_trials_confusion = dataloader.get_trials_label_with_confusion(task2_pred, tst_trials_label, test2train, tst_trials )
dev_EER_confusion = dataloader.calculate_EER_with_confusion(task1_pred[:,1],tst_trials_confusion)
print(f'time used = {time()-start:.3}')

Top 1 detector EER is 13.48% (Total confusion error is 402)
time used = 43.3
